In [2]:
import os
import torch
import torchvision


import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt

from PIL import Image

In /usr/remote/anaconda-3.7-2020-05-28/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /usr/remote/anaconda-3.7-2020-05-28/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /usr/remote/anaconda-3.7-2020-05-28/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /usr/remote/anaconda-3.7-2020-05-28/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /usr/

In [2]:
model=torchvision.models.resnet18(pretrained=True)

In [3]:
print('Trainable params: {}'.format(sum(p.numel() for p in model.parameters() if p.requires_grad)))

Trainable params: 11689512


In [4]:
model.fc = nn.Sequential(
    nn.Linear(in_features=512, out_features=15, bias=True),
    nn.Sigmoid())

model.layer4.requires_grad = False

for name, child in model.named_children():
    print(name, child)

conv1 Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
bn1 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
relu ReLU(inplace=True)
maxpool MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
layer1 Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1

In [5]:
n_epochs = 10
batch_size_train = 8# was 64
batch_size_test = 8
learning_rate = 0.001
momentum = 0.1
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [6]:
''' 
    NOTES: 
    - it's "No Finding" not "No findings"
    - it's "Pleural_Thickening" not "Pleural_thickening"
    - it's not "Nodule Mass", but rather "Nodule" and "Mass" separately
'''
disease_map = {"Atelectasis" : 0, "Consolidation" : 1, "Infiltration" : 2, "Pneumothorax": 3, "Edema": 4,
               "Emphysema": 5, "Fibrosis": 6, "Effusion" : 7, "Pneumonia" : 8, "Pleural_Thickening" : 9,
               "Cardiomegaly" : 10, "Nodule" : 11, "Mass" : 12, "Hernia" : 13, "No Finding" : 14 }

In [7]:
class GetLoader(torch.utils.data.Dataset):
    def __init__(self, data_root, labels_path, transforms=None):
        
        self.root = data_root
        self.transforms = transforms
    
        f = open(labels_path, 'r')
        datalist = f.readlines()
        f.close()
        
        
        self.len_data = len(datalist)

        self.img_paths = []
        self.img_labels = []
        
        self.cur_img_path = ''

        # csv file format:
        # Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],
        
        # example:
        # 00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143
        
        for data in datalist:
            data = data.split(',') # get csv line as list
            self.img_paths.append(data[0]) # image name is first in csv file
            
            diseases = data[1].split('|')
            one_hot = [0] * 15
            for d in diseases:
                hot_index = disease_map[d]
                one_hot[hot_index] = 1
            
            self.img_labels.append(torch.Tensor(one_hot)) # disease type is second in csv file

    def __getitem__(self, item):
        img_paths, labels = self.img_paths[item], self.img_labels[item]
        imgs = Image.open(os.path.join(self.root, img_paths)) # .convert('RGB')
        self.cur_img_path = os.path.join(self.root, img_paths)

        if self.transforms is not None:
            for t in self.transforms:
                imgs = t(imgs)

        return imgs, labels
    
    def get_img_path(self):
        return self.cur_img_path

    def __len__(self):
        return self.len_data

In [8]:
# locations from which to grab training / testing data
# image_root_train = os.path.join('.', 'data', 'sorted_for_use', 'train')
# image_root_test = os.path.join('.', 'data', 'sorted_for_use', 'test')

# label_path_train = os.path.join('.', 'data', 'sorted_for_use', 'train.csv')
# label_path_test = os.path.join('.', 'data', 'sorted_for_use', 'test.csv')

image_root_train = os.path.join('.', 'data_subset', 'train')
image_root_test = os.path.join('.', 'data_subset', 'test')

label_path_train = os.path.join('.', 'data_subset', 'train_subset.csv')
label_path_test = os.path.join('.', 'data_subset', 'test_subset.csv') 

print(label_path_train)
print(label_path_test)

# need a train and test dataset object
#torchvision.transforms.Resize((224, 224))
transforms = [torchvision.transforms.Grayscale(), torchvision.transforms.ToTensor()]
dataset_train = GetLoader(image_root_train, label_path_train, transforms=transforms)
dataset_test = GetLoader(image_root_test, label_path_test, transforms=transforms)

# these should just have the new dataset objects in them
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size_train, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size_test, shuffle=True, drop_last=True)

print(len(train_loader))
print(len(test_loader))

./data_subset/train_subset.csv
./data_subset/test_subset.csv
283
70


In [9]:
model = torch.nn.DataParallel(model)
model.cuda()
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                      momentum=momentum)
print('Trainable params: {}'.format(sum(p.numel() for p in model.parameters() if p.requires_grad)))

Trainable params: 11184207


In [10]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]
outputs = []

In [17]:
def train(epoch):
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):  # (output - actual )
        
        data = torch.cat((data, data, data), 1)
        #print(data.shape)
        
        if torch.cuda.is_available:
            data, target = data.cuda(), target.cuda()

        optimizer.zero_grad()
        output = model(data)
        outputs.append(output)
        
        loss_t = torch.nn.BCELoss()
        loss = loss_t(output, target)
        loss.backward()
        optimizer.step()
        
        #if(batch_idx == 1):
            #print(output)
        
        if batch_idx % log_interval == 0:
#             print('Train Epoch: {} \tLoss: {:.6f}'.format(
#                 epoch, loss.item()))            
            #print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                #epoch, batch_idx * len(data), len(train_loader.dataset),
                #100. * batch_idx / len(train_loader), loss.item()))
    
            train_losses.append(loss.item())
            train_counter.append(
            (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
            torch.save(model.state_dict(), './results')
            torch.save(optimizer.state_dict(), './results')

In [18]:
test_one = True

In [19]:
def test(test_one):
    
    model.eval()
    test_loss = 0
    correct = 0
    
    num_tested = 0
    
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            
            data = torch.cat((data, data, data), 1)
            #print(data.shape)
            
            if torch.cuda.is_available:
                data, target = data.cuda(), target.cuda()
                
            if batch_idx == 50:
                break;
            else:
                num_tested += len(data)
            
            output = model(data)
            loss_t = torch.nn.BCELoss()
            test_loss += loss_t(output, target).item()
            #print(output)
            pred = output.clone().cpu()
            target = target.cpu()
            #pred[pred > 0.5] = 1
            

            
            for p in pred:
                p = p.data

                for i in range(len(p)):
                    if p[i] > 0.5:
                        p[i] = 1
                    else:
                        p[i] = 0
                torch.tensor(p)
                
#             if test_one:
#                 print("printing correct ones:")
#                 for i in range(pred.shape[0]):
#                     #if torch.equal(pred[i], target[i]):
#                     print("-----------------------")
#                     print(torch.tensor(pred[i] == target.data[i]))
#                     print("output: " + str(output[0]))
#                     print("pred: " + str(pred[0]))
#                     print("target: " + str(target[0]))
#                 print("that's all")
#                 test_one = False
            


            correct = torch.sum(torch.tensor(pred == target)).sum()
        
    test_loss /= num_tested
    test_losses.append(test_loss)
    
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, num_tested,
    100. * correct / num_tested))

In [21]:
print("Starting Training")
test(test_one)
for epoch in range(1, n_epochs + 1):
    print("-----------Epoch " + str(epoch) + "----------------")
    train(epoch)
    test(test_one)

Starting Training


/usr/remote/anaconda-3.7-2020-05-28/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/remote/anaconda-3.7-2020-05-28/lib/python3.7/site-packages/ipykernel_launcher.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Test set: Avg. loss: 0.0475, Accuracy: 102/400 (26%)

-----------Epoch 1----------------


KeyboardInterrupt: 

In [3]:
plt.plot(test_losses)
plt.plot(train_losses)
plt.xlabel("epoch")
plt.ylabel("loss")
plt.figure()

NameError: name 'test_losses' is not defined